```{r}
library(magrittr)
library(quanteda)
library(memisc)
library(sweater)
```

```{r}
#| include: false
set.seed(721831)
```

## Empowerment statement

In this Methods Hub entry, I am going to show you how to test for implicit associations using word embeddings trained on a large language corpus.

## Cosine distance

Suppose we have the following corpus of 10 documents:

**Doc 1**: berlin is the capital of germany

**Doc 2**: paris is the capital of france

**Doc 3**: tokyo is the capital of japan

**Doc 4**: the cat is weird

**Doc 5**: berlin

**Doc 6**: paris is nice

**Doc 7**: paris is nice

**Doc 8**: paris is nice

**Doc 9**: paris is nice

**Doc 10**: berlin is weird

The unique token types are: "berlin", "is", "the", "capital", "of", "germany", "paris", "france", "tokyo", "japan", "cat", "weird", "nice". The representation of the above corpus as a document-term matrix is:

|    | berlin | is | the | capital | of | germany | paris | france | tokyo | japan | cat | weird | nice |
|----|--------|----|-----|---------|----|---------|-------|--------|-------|-------|-----|-------|------|
| 1  | 1      | 1  | 1   | 1       | 1  | 1       | 0     | 0      | 0     | 0     | 0   | 0     | 0    |
| 2  | 0      | 1  | 1   | 1       | 1  | 0       | 1     | 1      | 0     | 0     | 0   | 0     | 0    |
| 3  | 0      | 1  | 1   | 1       | 1  | 0       | 0     | 0      | 1     | 1     | 0   | 0     | 0    |
| 4  | 0      | 1  | 1   | 0       | 0  | 0       | 0     | 0      | 0     | 0     | 1   | 1     | 0    |
| 5  | 1      | 0  | 0   | 0       | 0  | 0       | 0     | 0      | 0     | 0     | 0   | 0     | 0    |
| 6  | 0      | 1  | 0   | 0       | 0  | 0       | 1     | 0      | 0     | 0     | 0   | 0     | 1    |
| 7  | 0      | 1  | 0   | 0       | 0  | 0       | 1     | 0      | 0     | 0     | 0   | 0     | 1    |
| 8  | 0      | 1  | 0   | 0       | 0  | 0       | 1     | 0      | 0     | 0     | 0   | 0     | 1    |
| 9  | 0      | 1  | 0   | 0       | 0  | 0       | 1     | 0      | 0     | 0     | 0   | 0     | 1    |
| 10 | 1      | 1  | 0   | 0       | 0  | 0       | 0     | 0      | 0     | 0     | 0   | 1     | 0    |

The row vector is the vector representation of the document. One way to compare the similarity between two **documents** is to calculate the cosine similarity. Cosine similarity between two vectors ($\mathbf{A}$, $\mathbf{B}$) is defined as:

$$
\cos(\theta) = {\mathbf{A} \cdot \mathbf{B} \over \|\mathbf{A}\| \|\mathbf{B}\|}
$$

For example, the cosine similarity between *Doc 1* `[1,1,1,1,1,1,0,0,0,0,0,0,0]` and *Doc 2* `[0,1,1,1,1,0,1,1,0,0,0,0,0]` is:

$$
\begin{align}
\mathbf{A} \cdot \mathbf{B} &= 1 \times 0 + 1 \times 1 + 1 \times 1 +1 \times 1 +1 \times 1 + 1 \times 0 + 0 \times 1 + 0 \times 1 + 0 \times 0 + 0 \times 0 + 0 \times 0 + 0 \times 0 + 0 \times 0 \\
&= 4\\
\|\mathbf{A}\| &= \sqrt{1^2 + 1^2 + 1^2 + 1^2 + 1^2 + 1^2 + 0^2 + 0^2 + 0^2 + 0^2 + 0^2 + 0^2 + 0^2} \\
&= \sqrt{6}\\
\|\mathbf{B}\| &= \sqrt{0^2 + 1^2 + 1^2 + 1^2 + 1^2 + 0^2 + 1^2 + 1^2 + 0^2 + 0^2 + 0^2 + 0^2 + 0^2} \\
&= \sqrt{6}\\
\cos(\theta) &= { 4 \over \sqrt{6} \times \sqrt{6}}\\
&= 0.\overline{6}
\end{align}
$$

In R, `lsa` [@lsarpackage] can be used to calculate cosine similarity.

```{r cosine1}
library(lsa)
doc1 <- c(1,1,1,1,1,1,0,0,0,0,0,0,0)
doc2 <- c(0,1,1,1,1,0,1,1,0,0,0,0,0)
cosine(doc1, doc2)
```

Or using `quanteda` [@benoit:2018]

```{r quanteda}
require(quanteda)
docs <- c("berlin is the capital of germany",
          "paris is the capital of france",
          "tokyo is the capital of japan",
          "the cat is weird",
          "berlin",
          "paris is nice",
          "paris is nice",
          "paris is nice",
          "paris is nice",
          "berlin is weird")
docs_dtm <- corpus(docs) %>% tokens %>% dfm()
docs_dtm
```

```{r quantedacosine}
cosine(as.vector(docs_dtm[1,]), as.vector(docs_dtm[2,]))
```

The cosine similarity between *Doc 1* and *Doc 6* is much lower, as there is just one common word *"is"*.

```{r cosine2}
doc6 <- c(0,1,0,0,0,0,1,0,0,0,0,0,1)
##or
##cosine(as.vector(docs_dtm[1,]), as.vector(docs_dtm[6,]))
cosine(doc1, doc6)
```

## One-hot vectors

In the traditional "bag of words" text representation, a word is presented as a so-called "one-hot" vector. In the above example, the fifth document has just one word of "berlin". This document is represented as row vector `[1,0,0,0,0,0,0,0,0,0,0,0,0]` in the document-term matrix. This vector is sparse (many zeros). One can also reason it as the "one-hot" vector representation of the word "berlin", because there is exactly one instance of "1" in the entire vector. One-hot vector representation of word is not very useful for comparison between **words**. For example, cosine similarity of "one-hot" vectors of two different words is always 0.

```{r allzero}
## comparing "berlin" and "paris"
cosine(c(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
       c(0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0))
## comparing "berlin" and "cat"
cosine(c(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
       c(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0))
## comparing "berlin" and "nice"
cosine(c(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
       c(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1))
## comparing "paris" and "nice"
cosine(c(0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0),
       c(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1))
```

However, we should anticipate that "berlin" and "paris" should be more similar in latent meaning (at least both of them are described in the context of "capital") than "berlin" and "cat".

## (Dense) word vectors

An improvement to this is to make them "dense" word vectors. One way is to train word embeddings on the corpus in order to generate dense word vectors. Word embeddings can capture the **distributional semantics** of words: words that are used and occur in the same contexts tend to purport similar meanings.

In the following example, the GLoVe embeddings are trained [@pennington:2014:G]. The GLoVe algorithm is based on the weighted feature-cooccurrence matrix (FCM) representation of the corpus, which can be generated by quanteda [@benoit:2018].

There are three parameters needed to be specified: `WINDOWS_SIZE`, `RANK`, `LEARNING_RATE`. `WINDOW_SIZE` determines how close two words are is being considered as related. `RANK` determines the output length of the word vector. `LEARNING_RATE` determines the learning rate of the algorithm. 

The FCM is a square matrix with each cell represents how likely two words are cooccured.

```{r fcm}
#| warning: false
WINDOW_SIZE <- 3

weighted_fcm_corpus <- corpus(docs) %>% tokens() %>%
    fcm(window = WINDOW_SIZE, weights = 1 / seq_len(WINDOW_SIZE),
        count = "weighted", context = "window", tri = TRUE)
weighted_fcm_corpus
```

```{r train}
#| message: false
#| output: false
library(rsparse)

RANK <- 5
LEARNING_RATE <- 0.05

glove <- GloVe$new(rank = RANK, x_max = RANK / 2, learning_rate = LEARNING_RATE)

wv_main <- glove$fit_transform(weighted_fcm_corpus, n_iter = 100,
                               convergence_tol = 0.01, n_threads = 8)

wv_context <- glove$components
dense_vectors <- wv_main + t(wv_context)
```

The (dense) word vectors are, as the name suggested, dense. The dimension is equal to `RANK`. We can also think about the following matrix as a multidimensional *word embedding space*.

```{r dense}
dense_vectors
```

And the row vectors can be compared using cosine similarity. Now, we can see that the similarity between "berlin" and "paris" is higher than "berlin" and "cat".

```{r dist}
cosine(dense_vectors["berlin",], dense_vectors["paris",])
cosine(dense_vectors["berlin",], dense_vectors["cat",])
```

## Target words and attribute words

Suppose "paris", "tokyo", and "berlin" are the words we are interested in. We can called these words *target words*. We can determine how these words are similar to some other words in the word embeddings space. Suppose we set "nice" to be the attribute word. By determining the similarities between the target words and attribute words, we can see the implicit association between words. In this example, we can see that "paris" is highly associated with "nice"; but not "tokyo" and "berlin".

```{r sa}
cosine(dense_vectors["paris",], dense_vectors["nice",])
cosine(dense_vectors["tokyo",], dense_vectors["nice",])
cosine(dense_vectors["berlin",], dense_vectors["nice",])
```

## Pretrained word embeddings

We can train our own word embeddings. But there are also several pretrained word embeddings trained on large corpora available for download.

* [word2vec](https://huggingface.co/fse/word2vec-google-news-300) trained on Google News 
* [GLoVe](https://nlp.stanford.edu/projects/glove/) trained on Wikipedia, Common Crawl, and Gigaword
* [fastText](https://fasttext.cc/docs/en/crawl-vectors.html) trained on Wikipedia and Common Crawl

In the following example, we will use the pretrained GLoVe word embeddings to replicate the findings by @caliskan:2017:S and pretrained word2vec word embeddings to replicate the findings from @garg:2018:W. The R package `sweater` [@chan:2022] can be used to read the downloaded word embedding file.

```r
require(sweater)
glove <- read_word2vec("glove.840B.300d.txt")
```

The package also provides a subset of the word embeddings called `glove_math`.

## Query

`sweater` use the concept of query to look for associations.

A query requires two sets of words: Target words ($\mathcal{S}$, $\mathcal{T}$) and Attribute words ($\mathcal{A}$, $\mathcal{B}$). The package provides different methods and they require different combinations of $\mathcal{S}$, $\mathcal{T}$, $\mathcal{A}$, and $\mathcal{B}$.

| Method                           | Target words                 | Attribute words              |
|----------------------------------|------------------------------|------------------------------|
| Mean Average Cosine Similarity   | $\mathcal{S}$                | $\mathcal{A}$                |
| Relative Norm Distance           | $\mathcal{S}$                | $\mathcal{A}$, $\mathcal{B}$ |
| Relative Negative Sentiment Bias | $\mathcal{S}$                | $\mathcal{A}$, $\mathcal{B}$ |
| SemAxis                          | $\mathcal{S}$                | $\mathcal{A}$, $\mathcal{B}$ |
| Normalized Association Score     | $\mathcal{S}$                | $\mathcal{A}$, $\mathcal{B}$ |
| Embedding Coherence Test         | $\mathcal{S}$                | $\mathcal{A}$, $\mathcal{B}$ |
| Word Embedding Association Test  | $\mathcal{S}$, $\mathcal{T}$ | $\mathcal{A}$, $\mathcal{B}$ |

All methods use the same `query` function.

```r
query(w, S_words, T_words, A_words, B_words, method = "guess", verbose = FALSE)
```

## Case study: Gender biases in word embeddings

### Word Embedding Association Test

Word Embedding Association Test (WEAT) [@caliskan:2017:S] requires all four wordsets of $\mathcal{S}$, $\mathcal{T}$, $\mathcal{A}$, and $\mathcal{B}$. The method is modeled after the Implicit Association Test (IAT) and it measures the relative strength of $\mathcal{S}$'s association with $\mathcal{A}$ to $\mathcal{B}$ against the same of $\mathcal{T}$. 

```{r}
#| include: false
detach("package:lsa", unload=TRUE)
```

```{r caliskan}
require(sweater)
S <- c("math", "algebra", "geometry", "calculus", "equations", "computation",
       "numbers", "addition")
T <- c("poetry", "art", "dance", "literature", "novel", "symphony", "drama",
       "sculpture")
A <- c("male", "man", "boy", "brother", "he", "him", "his", "son")
B <- c("female", "woman", "girl", "sister", "she", "her", "hers", "daughter")
sw <- query(glove_math, S, T, A, B)
sw
```

The effect size can be interpreted the same way as Cohen's d. In this case, the effect size is positive, indicating the math-related concepts are more associated with male than female, whereas art-related concepts are more associated with female than male.

### Relative Norm Distance

@garg:2018:W use Relative Norm Distance to quantify gender biases of occupation words in the pretrained Google News Word Embeddings. The method requires one set of target words and two sets of attribute words.

```{r garg2}
require(sweater)
S1 <- c("janitor", "statistician", "midwife", "bailiff", "auctioneer", 
"photographer", "geologist", "shoemaker", "athlete", "cashier", 
"dancer", "housekeeper", "accountant", "physicist", "gardener", 
"dentist", "weaver", "blacksmith", "psychologist", "supervisor", 
"mathematician", "surveyor", "tailor", "designer", "economist", 
"mechanic", "laborer", "postmaster", "broker", "chemist", "librarian", 
"attendant", "clerical", "musician", "porter", "scientist", "carpenter", 
"sailor", "instructor", "sheriff", "pilot", "inspector", "mason", 
"baker", "administrator", "architect", "collector", "operator", 
"surgeon", "driver", "painter", "conductor", "nurse", "cook", 
"engineer", "retired", "sales", "lawyer", "clergy", "physician", 
"farmer", "clerk", "manager", "guard", "artist", "smith", "official", 
"police", "doctor", "professor", "student", "judge", "teacher", 
"author", "secretary", "soldier")

A1 <- c("he", "son", "his", "him", "father", "man", "boy", "himself", 
"male", "brother", "sons", "fathers", "men", "boys", "males", 
"brothers", "uncle", "uncles", "nephew", "nephews")

B1 <- c("she", "daughter", "hers", "her", "mother", "woman", "girl", 
"herself", "female", "sister", "daughters", "mothers", "women", 
"girls", "females", "sisters", "aunt", "aunts", "niece", "nieces"
)
res <- query(googlenews, S_words = S1, A_words = A1, B_words = B1)
res
```

The more positive effect size indicates that words in $\mathcal{S}$ are more associated with $\mathcal{B}$. As the effect size is negative, it indicates that the concept of occupation is more associated with $\mathcal{B}$, i.e. male.

We can also produce a visualization to study the differences among target words. Target words such as "nurse", "midwife", and "housekeeper" are more associated with female than male.

```{r gargvis}
#| fig-height: 10
plot(res)
```

## Conclusion

In this entry, I showed you how the quantification of implicit association among words works. The R package `sweater` was introduced for this task.

## References
